# Learning to optimize the $a$ and $b$ parameters of a function $a^2+b$ to minimize it

In [1]:
from pathlib import Path
from ray import tune, air

In [2]:
def objective(config):
    score = config["a"] ** 2 + config["b"]
    return {"score": score}

In [3]:
config_example1 = {"a": 5, "b": 7}
print(
    f"a={config_example1['a']}, b={config_example1['b']}, result={objective(config_example1)['score']}"
)
config_example2 = {"a": -4, "b": -1}
print(
    f"a={config_example2['a']}, b={config_example2['b']}, result={objective(config_example2)['score']}"
)
config_example3 = {"a": 7, "b": 5}
print(
    f"a={config_example3['a']}, b={config_example3['b']}, result={objective(config_example3)['score']}"
)

a=5, b=7, result=32
a=-4, b=-1, result=15
a=7, b=5, result=54


In [4]:
search_space = {"a": tune.uniform(0, 100), "b": tune.choice([0, 10, 500, 21])}

print(f"Variable a sample:{search_space['a'].sample()}")
print(f"Variable b sample:{search_space['b'].sample()}")

# Execute this cell multiple times to see different samples

Variable a sample:1.5353918864193683
Variable b sample:0


There are different search space types available in the [Ray Tune API](https://docs.ray.io/en/latest/tune/api/search_space.html).

## Configuring/running the Tuner to optimize the function considering the defined parameter space

It is the same tune.Tuner structure we used in the [lesson 1 activity 2](../lesson_1/2-simple_ppo_agent_to_environment.ipynb) to implement the PPO Rl agent, but here we are utilizing it without considering RL.

In [5]:
stop = {
    "training_iteration": 1000,
}
store_results_path = str(Path("./ray_results/").resolve()) + "/nb_1/"
agent_name = "random_agent"
tuner = tune.Tuner(
    objective,
    param_space=search_space,
    run_config=air.RunConfig(
        stop=stop,
        verbose=2,
        storage_path=store_results_path,
        name=agent_name,
    ),
)
results = tuner.fit()
print(results.get_best_result(metric="score", mode="min").config)

2024-11-29 17:01:34,727	INFO worker.py:1783 -- Started a local Ray instance.
2024-11-29 17:01:35,270	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2024-11-29 17:01:35,272	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,score
objective_97307_00000,5004.4


2024-11-29 17:01:37,873	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/lasse/ray_minicourse/lesson_2/ray_results/nb_1/random_agent' in 0.0029s.
2024-11-29 17:01:37,877	INFO tune.py:1041 -- Total run time: 2.61 seconds (1.27 seconds for the tuning loop).


{'a': 67.11482071676609, 'b': 500}
